Duplicate images into new folder

In [4]:
import os
import shutil

# Input image path (relative path)
input_image_path = "dtd_folder/dtd/images/honeycombed/honeycombed_0003.jpg"  # Replace with your image path

# Output directory
output_dir = "dtd_folder/dtd/images/curatedHoneycomb_WGAN"
os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

# Number of duplicates
num_duplicates = 120

# Loop to create duplicates with unique filenames
for i in range(1, num_duplicates + 1):
    output_file = os.path.join(output_dir, f"honeycomb_{i}.jpg")
    shutil.copy(input_image_path, output_file)

print(f"Duplicated the image {num_duplicates} times in the folder '{output_dir}'.")

Duplicated the image 120 times in the folder 'dtd_folder/dtd/images/curatedHoneycomb_WGAN'.


In [5]:
import cv2
import homography
# Input image path (relative path)
input_dir = "dtd_folder/dtd/images/curatedHoneycomb_WGAN"

# Output directory
output_dir = "dtd_folder/dtd/images/curatedHoneycombHOMOGRAPHY_WGAN"
os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

# Loop to create duplicates with unique filenames
for i in range(1, 21):
    output_file = os.path.join(output_dir, f"honeycomb_{i}.jpg")
    input_file = os.path.join(input_dir, f'honeycomb_{i}.jpg')
    img = cv2.imread(input_file)
    H = homography.generate_homography_warp(img.shape[:2])
    img_out = homography.apply_homography(img, H)
    cv2.imwrite(output_file, img_out)
    
print(f"Duplicated the image {num_duplicates} times in the folder '{output_dir}'.")

Duplicated the image 120 times in the folder 'dtd_folder/dtd/images/curatedHoneycombHOMOGRAPHY_WGAN'.


In [6]:
from utils import load_and_preprocess_images
from config import Config
config = Config()

images_dir = 'C:/Users/ASC_Student/Documents/GitHub/GAN-Texture-Synthesis/dtd_folder/dtd/images/curatedHoneycomb_WGAN'
images_dir_warped = 'C:/Users/ASC_Student/Documents/GitHub/GAN-Texture-Synthesis/dtd_folder/dtd/images/curatedChequeredHOMOGRAPHY_WGAN'
print("Loading images!")

dataset = load_and_preprocess_images(images_dir, target_size=(160, 160), batch_size=config.batch_size)

# for homography testing (separate pipeline, not in GAN)
# dataset_warped = load_and_preprocess_images(images_dir_warped, target_size=(160, 160), batch_size=config.batch_size)

# combined_dataset = dataset.concatenate(dataset_warped)

Loading images!
Found 120 files.


C:\Users\ASC_Student\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tensorflow\python\data\ops\structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [ ]:
from trainWGAN import train 

print("Starting PSGAN training...wish me luck")
loss_g, loss_d = train(dataset, epochs=config.epoch_count)
print("Training complete! Check 'samples/' for generated images and 'models/' for saved models.")